自然言語処理関連の特徴量作成用notebook。

Output Files:

ALL_w2v.csv, ALL_svd_fastText.csv, ALL_pca_fastText.csv, ALL_svd_SciBERT.csv, 

w2v.csv, svd_fastText.csv, pca_fastText.csv, svd_SciBERT.csv, tfidf.csv

# Libraries

In [ ]:
!pip install texthero

In [ ]:
!pip install fasttext

In [ ]:
!pip install transformers

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import random
import os
import time
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import seaborn as sns
import re
import itertools
import collections
import pickle
import texthero as hero
import torch
import transformers
from transformers import BertTokenizer
from contextlib import contextmanager
from pathlib import Path
from typing import Optional
from fasttext import load_model
from gensim.models import word2vec, KeyedVectors
from geopy.geocoders import Nominatim
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm

tqdm.pandas()

# Utils

In [ ]:
@contextmanager
def timer(name: str):
    t0 = time.time()
    print(f"[{name}] start")
    yield
    msg = f"[{name}] done in {time.time() - t0:.0f} s"
    print(msg)
    
    
def set_seed(seed=42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    
SEED = 42
set_seed(SEED)

# Data Loading

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd "drive/My Drive/probspace_citations"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/probspace_citations


In [ ]:
DATADIR = Path('input')
OUTDIR = Path('features')

In [ ]:
with timer("Load ALL_df"):
    ALL_df = pd.read_pickle(DATADIR / 'ALL_df.pkl') # 全てのtrain/testを結合したデータ

[Load ALL_df] start
[Load ALL_df] done in 31 s


In [ ]:
with timer("Load df"):
    df = pd.read_pickle(DATADIR / 'df.pkl') # citesの値が与えられたtrainとtestを結合したデータ

[Load df] start
[Load df] done in 4 s


# ALL_df Features

citesの無いtrainデータ含めて作成した特徴量

## Word2Vec

In [ ]:
def make_id_feat(x):
    try: return re.search('(.*)/(.*)', x).group(1)
    except: return x[:4]
def make_doi_feat(x):
    try: return x.split('/')[0]
    except: return np.nan

ALL_df['id_feat'] = ALL_df['id'].apply(make_id_feat)
ALL_df['doi_feat'] = ALL_df['doi'].apply(make_doi_feat)
ALL_df['categories_space_list'] = ALL_df['categories'].apply(lambda x: list(x.split()))
ALL_df['categories_comma_list'] = ALL_df['categories'].apply(lambda x: list(re.split('[. ]', x)))

In [ ]:
id_doi_df = pd.concat([ALL_df['id'],(ALL_df['id_feat'] + ' ' +  ALL_df['doi_feat']).apply(lambda x: list(x.split()))],axis=1).rename(columns={0: 'target'})
categories_space_df = pd.concat([ALL_df['id'],ALL_df['categories_space_list']],axis=1).rename(columns={'categories_space_list': 'target'})
categories_comma_df = pd.concat([ALL_df['id'],ALL_df['categories_comma_list']],axis=1).rename(columns={'categories_comma_list': 'target'})
id_doi_categories_space_df = pd.concat([ALL_df['id'],id_doi_df['target'] + categories_space_df['target']],axis=1).rename(columns={0: 'target'})

In [ ]:
model_size = {
    'id_doi': 16,
    'categories_space': 16,
    'categories_comma': 32,
    'id_doi_categories_space': 64,
}

n_iter = 100
w2v_dfs = []
for _df, _df_name in zip(
        [id_doi_df, categories_space_df, categories_comma_df, id_doi_categories_space_df],
        ['id_doi', 'categories_space', 'categories_comma', 'id_doi_categories_space']
    ):

    with timer(f"Creating w2v for {_df_name}"):
        # Word2Vecの学習
        w2v_model = word2vec.Word2Vec(_df['target'].values.tolist(),
                                    size=model_size[_df_name],
                                    min_count=1,
                                    window=100,
                                    workers=1,
                                    iter=n_iter)

    with timer(f"Getting document vector for {_df_name}"):
        # 各文章ごとにそれぞれの単語をベクトル表現に直し、平均をとって文章ベクトルにする
        sentence_vectors = _df['target'].progress_apply(lambda x: np.mean([w2v_model.wv[e] for e in x], axis=0))
        sentence_vectors = np.vstack([x for x in sentence_vectors])
        sentence_vector_df = pd.DataFrame(sentence_vectors,
                                        columns=[f"ALL_{_df_name}_w2v_{i}" for i in range(model_size[_df_name])])
        sentence_vector_df.index = _df['id']
        w2v_dfs.append(sentence_vector_df)

[Creating w2v for id_doi] start


  1%|          | 6881/910608 [00:00<00:25, 35497.37it/s]

[Creating w2v for id_doi] done in 210 s
[Getting document vector for id_doi] start


100%|██████████| 910608/910608 [00:25<00:00, 36067.55it/s]


[Getting document vector for id_doi] done in 27 s
[Creating w2v for categories_space] start


...

[Getting document vector for categories_comma] done in 25 s
[Creating w2v for id_doi_categories_space] start


  0%|          | 3520/910608 [00:00<00:25, 35196.76it/s]

[Creating w2v for id_doi_categories_space] done in 362 s
[Getting document vector for id_doi_categories_space] start


100%|██████████| 910608/910608 [00:24<00:00, 37431.16it/s]


[Getting document vector for id_doi_categories_space] done in 27 s


In [ ]:
ALL_w2v = df[['id']]
for i in range(4):
    ALL_w2v = pd.merge(ALL_w2v, w2v_dfs[i], on='id', how='left')

In [ ]:
ALL_w2v.to_csv(OUTDIR / 'ALL_w2v.csv', index=False)

## fastText features

fastTextの学習済モデルを用いた特徴量作成

In [ ]:
model_en = load_model("cc.en.300.bin")

In [ ]:
def text_cleaning(raw_text):
    clean_text = hero.clean(raw_text, pipeline=[
        hero.preprocessing.fillna, # 欠損埋め
        hero.preprocessing.lowercase, # 小文字への統一
        hero.preprocessing.remove_digits, # 数字の削除
        hero.preprocessing.remove_punctuation, # 句読点の削除
        hero.preprocessing.remove_diacritics, # ダイアクリティカルマーク（発音区別符号。àやéなど）の削除
        hero.preprocessing.remove_stopwords, # ストップワードの除去
        hero.preprocessing.remove_whitespace, # スペースの削除
        hero.preprocessing.remove_brackets,
    ])
    return clean_text

In [ ]:
ALL_df['abstract'] = text_cleaning(ALL_df['abstract'])
ALL_df['title'] = text_cleaning(ALL_df['title'])
ALL_df['comments'] = text_cleaning(ALL_df['comments'])

In [ ]:
ALL_svd_fastText = pd.DataFrame(ALL_df["id"])
ALL_pca_fastText = pd.DataFrame(ALL_df["id"])
ALL_svd_fastText

,id
0,hep-ph/9902295
1,1403.7138
2,1405.5857
3,1807.01034
4,1905.05921
...,...
910603,1210.4112
910604,1701.03465
910605,1709.10428
910606,gr-qc/9803020


### TruncatedSVD

In [ ]:
with timer("ALL SVD abstract fastText"):
    X = ALL_df["abstract"].progress_apply(lambda x: model_en.get_sentence_vector(x.replace("\n", "")))
    X = np.stack(X.values)
    svd = TruncatedSVD(n_components=64, random_state=SEED)
    X = svd.fit_transform(X)
    X_df = pd.DataFrame(X, columns=[f"ALL_SVD_abstract_fastText_{i}" for i in range(X.shape[1])])
    X_df["id"] = ALL_df["id"]
    ALL_svd_fastText = ALL_svd_fastText.merge(X_df, on="id", how="left")

  0%|          | 570/910608 [00:00<06:25, 2361.70it/s]

[ALL SVD abstract fastText] start


100%|██████████| 910608/910608 [04:17<00:00, 3540.78it/s]


[ALL SVD abstract fastText] done in 278 s


In [ ]:
with timer("ALL SVD title fastText"):
    X = ALL_df["title"].progress_apply(lambda x: model_en.get_sentence_vector(x.replace("\n", "")))
    X = np.stack(X.values)
    svd = TruncatedSVD(n_components=32, random_state=SEED)
    X = svd.fit_transform(X)
    X_df = pd.DataFrame(X, columns=[f"ALL_SVD_title_fastText_{i}" for i in range(X.shape[1])])
    X_df["id"] = ALL_df["id"]
    ALL_svd_fastText = ALL_svd_fastText.merge(X_df, on="id", how="left")

  0%|          | 4275/910608 [00:00<00:43, 20980.43it/s]

[ALL SVD title fastText] start


100%|██████████| 910608/910608 [00:39<00:00, 23157.36it/s]


[ALL SVD title fastText] done in 54 s


In [ ]:
with timer("ALL SVD comments fastText"):
    X = ALL_df["comments"].progress_apply(lambda x: model_en.get_sentence_vector(x.replace("\n", "")))
    X = np.stack(X.values)
    svd = TruncatedSVD(n_components=16, random_state=SEED)
    X = svd.fit_transform(X)
    X_df = pd.DataFrame(X, columns=[f"ALL_SVD_comments_fastText_{i}" for i in range(X.shape[1])])
    X_df["id"] = ALL_df["id"]
    ALL_svd_fastText = ALL_svd_fastText.merge(X_df, on="id", how="left")

  0%|          | 3531/910608 [00:00<00:25, 35305.15it/s]

[ALL SVD comments fastText] start


100%|██████████| 910608/910608 [00:25<00:00, 36104.23it/s]


[ALL SVD comments fastText] done in 36 s


In [ ]:
ALL_svd_fastText

,id,ALL_SVD_abstract_fastText_0,ALL_SVD_abstract_fastText_1,ALL_SVD_abstract_fastText_2,ALL_SVD_abstract_fastText_3,ALL_SVD_abstract_fastText_4,ALL_SVD_abstract_fastText_5,ALL_SVD_abstract_fastText_6,ALL_SVD_abstract_fastText_7,ALL_SVD_abstract_fastText_8,ALL_SVD_abstract_fastText_9,ALL_SVD_abstract_fastText_10,ALL_SVD_abstract_fastText_11,ALL_SVD_abstract_fastText_12,ALL_SVD_abstract_fastText_13,ALL_SVD_abstract_fastText_14,ALL_SVD_abstract_fastText_15,ALL_SVD_abstract_fastText_16,ALL_SVD_abstract_fastText_17,ALL_SVD_abstract_fastText_18,ALL_SVD_abstract_fastText_19,ALL_SVD_abstract_fastText_20,ALL_SVD_abstract_fastText_21,ALL_SVD_abstract_fastText_22,ALL_SVD_abstract_fastText_23,ALL_SVD_abstract_fastText_24,ALL_SVD_abstract_fastText_25,ALL_SVD_abstract_fastText_26,ALL_SVD_abstract_fastText_27,ALL_SVD_abstract_fastText_28,ALL_SVD_abstract_fastText_29,ALL_SVD_abstract_fastText_30,ALL_SVD_abstract_fastText_31,ALL_SVD_abstract_fastText_32,ALL_SVD_abstract_fastText_33,ALL_SVD_abstract_fastText_34,ALL_SVD_abstract_fastText_35,ALL_SVD_abstract_fastText_36,ALL_SVD_abstract_fastText_37,ALL_SVD_abstract_fastText_38,...,ALL_SVD_title_fastText_8,ALL_SVD_title_fastText_9,ALL_SVD_title_fastText_10,ALL_SVD_title_fastText_11,ALL_SVD_title_fastText_12,ALL_SVD_title_fastText_13,ALL_SVD_title_fastText_14,ALL_SVD_title_fastText_15,ALL_SVD_title_fastText_16,ALL_SVD_title_fastText_17,ALL_SVD_title_fastText_18,ALL_SVD_title_fastText_19,ALL_SVD_title_fastText_20,ALL_SVD_title_fastText_21,ALL_SVD_title_fastText_22,ALL_SVD_title_fastText_23,ALL_SVD_title_fastText_24,ALL_SVD_title_fastText_25,ALL_SVD_title_fastText_26,ALL_SVD_title_fastText_27,ALL_SVD_title_fastText_28,ALL_SVD_title_fastText_29,ALL_SVD_title_fastText_30,ALL_SVD_title_fastText_31,ALL_SVD_comments_fastText_0,ALL_SVD_comments_fastText_1,ALL_SVD_comments_fastText_2,ALL_SVD_comments_fastText_3,ALL_SVD_comments_fastText_4,ALL_SVD_comments_fastText_5,ALL_SVD_comments_fastText_6,ALL_SVD_comments_fastText_7,ALL_SVD_comments_fastText_8,ALL_SVD_comments_fastText_9,ALL_SVD_comments_fastText_10,ALL_SVD_comments_fastText_11,ALL_SVD_comments_fastText_12,ALL_SVD_comments_fastText_13,ALL_SVD_comments_fastText_14,ALL_SVD_comments_fastText_15
0,hep-ph/9902295,0.330791,-0.040681,-0.036119,0.013659,-0.016790,0.087835,0.048660,0.039931,0.031968,-0.024824,0.003808,0.042545,-0.005603,-0.050364,0.022270,0.033589,0.019019,0.101123,0.000044,0.013913,-0.020021,-0.023491,-0.030688,-0.015847,0.032511,0.028007,-0.001736,0.004960,-0.010098,0.045267,0.027386,-0.009925,-0.021486,-0.044623,0.013781,-0.013956,-0.039226,0.010656,0.008050,...,0.064884,-0.009866,0.061122,0.042080,-0.121580,-0.005575,0.059404,-0.057897,0.059150,-0.018576,-0.070641,0.018830,0.096663,-0.139469,-0.035567,-0.047924,0.010380,0.011918,0.026518,0.082314,-0.014170,-0.053461,-0.009393,0.009566,2.489214e-01,1.867214e-01,5.922201e-03,-4.138458e-02,0.117438,0.101735,0.010537,0.030196,-0.008867,-0.028349,-0.014525,-0.031311,-0.022195,-0.021503,-0.012576,0.020626
1,1403.7138,0.363122,0.029175,-0.061232,-0.003548,0.012257,-0.057539,0.032653,-0.012248,0.045867,-0.011151,0.008271,0.055627,0.030200,0.031697,0.025476,0.025927,0.001187,0.001148,-0.011435,-0.046066,-0.003005,-0.004352,0.001753,0.019525,-0.009865,0.005342,-0.000365,-0.000570,0.009256,0.010934,-0.005433,-0.000334,0.005619,-0.003421,-0.006042,-0.001026,0.032405,-0.018932,-0.005383,...,0.030021,-0.007932,-0.031351,-0.072215,-0.029012,0.006704,-0.016811,0.007148,-0.031679,-0.050298,0.044642,0.034284,-0.008809,0.004279,0.057215,0.043713,-0.015236,-0.024644,-0.000256,0.021189,-0.042301,-0.012851,-0.020254,-0.001741,4.625807e-01,1.755179e-01,1.996905e-02,1.759009e-01,-0.030985,-0.021186,-0.013829,0.027517,-0.017860,0.048070,-0.044942,0.009950,0.056432,0.068888,0.023235,0.017437
2,1405.5857,0.349692,0.035704,-0.011180,0.033973,-0.006480,0.029008,-0.046002,-0.008810,-0.014794,-0.011983,0.010968,0.002813,0.026181,-0.017254,-0.008956,-0.002908,-0.015610,-0.005823,-0.009052,0.014030,0.007281,0.018405,0.0021

In [ ]:
ALL_svd_fastText.to_csv(OUTDIR / 'ALL_svd_fastText.csv', index=False)

### PCA

In [ ]:
with timer("ALL PCA abstract fastText"):
    X = ALL_df["abstract"].progress_apply(lambda x: model_en.get_sentence_vector(x.replace("\n", "")))
    X = np.stack(X.values)
    pca = PCA(n_components=64, random_state=SEED)
    X = pca.fit_transform(X)
    X_df = pd.DataFrame(X, columns=[f"PCA_abstract_fastText_{i}" for i in range(X.shape[1])])
    X_df["id"] = ALL_df["id"]
    ALL_pca_fastText = ALL_pca_fastText.merge(X_df, on="id", how="left")

  0%|          | 566/910608 [00:00<06:20, 2392.74it/s]

[ALL PCA abstract fastText] start


100%|██████████| 910608/910608 [04:28<00:00, 3389.19it/s]


[ALL PCA abstract fastText] done in 289 s


In [ ]:
with timer("ALL PCA title fastText"):
    X = ALL_df["title"].progress_apply(lambda x: model_en.get_sentence_vector(x.replace("\n", "")))
    X = np.stack(X.values)
    pca = PCA(n_components=32, random_state=SEED)
    X = pca.fit_transform(X)
    X_df = pd.DataFrame(X, columns=[f"PCA_title_fastText_{i}" for i in range(X.shape[1])])
    X_df["id"] = ALL_df["id"]
    ALL_pca_fastText = ALL_pca_fastText.merge(X_df, on="id", how="left")

  1%|          | 4621/910608 [00:00<00:38, 23501.15it/s]

[ALL PCA title fastText] start


100%|██████████| 910608/910608 [00:41<00:00, 22125.75it/s]


[ALL PCA title fastText] done in 54 s


In [ ]:
with timer("ALL PCA comments fastText"):
    X = ALL_df["comments"].progress_apply(lambda x: model_en.get_sentence_vector(x.replace("\n", "")))
    X = np.stack(X.values)
    pca = PCA(n_components=16, random_state=SEED)
    X = pca.fit_transform(X)
    X_df = pd.DataFrame(X, columns=[f"PCA_comments_fastText_{i}" for i in range(X.shape[1])])
    X_df["id"] = ALL_df["id"]
    ALL_pca_fastText = ALL_pca_fastText.merge(X_df, on="id", how="left")

  0%|          | 2384/910608 [00:00<00:38, 23839.97it/s]

[ALL PCA comments fastText] start


100%|██████████| 910608/910608 [00:25<00:00, 35604.55it/s]


[ALL PCA comments fastText] done in 39 s


In [ ]:
ALL_pca_fastText

,id,PCA_abstract_fastText_0,PCA_abstract_fastText_1,PCA_abstract_fastText_2,PCA_abstract_fastText_3,PCA_abstract_fastText_4,PCA_abstract_fastText_5,PCA_abstract_fastText_6,PCA_abstract_fastText_7,PCA_abstract_fastText_8,PCA_abstract_fastText_9,PCA_abstract_fastText_10,PCA_abstract_fastText_11,PCA_abstract_fastText_12,PCA_abstract_fastText_13,PCA_abstract_fastText_14,PCA_abstract_fastText_15,PCA_abstract_fastText_16,PCA_abstract_fastText_17,PCA_abstract_fastText_18,PCA_abstract_fastText_19,PCA_abstract_fastText_20,PCA_abstract_fastText_21,PCA_abstract_fastText_22,PCA_abstract_fastText_23,PCA_abstract_fastText_24,PCA_abstract_fastText_25,PCA_abstract_fastText_26,PCA_abstract_fastText_27,PCA_abstract_fastText_28,PCA_abstract_fastText_29,PCA_abstract_fastText_30,PCA_abstract_fastText_31,PCA_abstract_fastText_32,PCA_abstract_fastText_33,PCA_abstract_fastText_34,PCA_abstract_fastText_35,PCA_abstract_fastText_36,PCA_abstract_fastText_37,PCA_abstract_fastText_38,...,PCA_title_fastText_8,PCA_title_fastText_9,PCA_title_fastText_10,PCA_title_fastText_11,PCA_title_fastText_12,PCA_title_fastText_13,PCA_title_fastText_14,PCA_title_fastText_15,PCA_title_fastText_16,PCA_title_fastText_17,PCA_title_fastText_18,PCA_title_fastText_19,PCA_title_fastText_20,PCA_title_fastText_21,PCA_title_fastText_22,PCA_title_fastText_23,PCA_title_fastText_24,PCA_title_fastText_25,PCA_title_fastText_26,PCA_title_fastText_27,PCA_title_fastText_28,PCA_title_fastText_29,PCA_title_fastText_30,PCA_title_fastText_31,PCA_comments_fastText_0,PCA_comments_fastText_1,PCA_comments_fastText_2,PCA_comments_fastText_3,PCA_comments_fastText_4,PCA_comments_fastText_5,PCA_comments_fastText_6,PCA_comments_fastText_7,PCA_comments_fastText_8,PCA_comments_fastText_9,PCA_comments_fastText_10,PCA_comments_fastText_11,PCA_comments_fastText_12,PCA_comments_fastText_13,PCA_comments_fastText_14,PCA_comments_fastText_15
0,hep-ph/9902295,-0.034009,-0.027467,0.034190,-0.020338,0.071076,0.079460,0.032222,0.030883,-0.028977,0.002730,0.042287,-0.007282,-0.045794,0.044792,0.033063,0.030043,0.080763,0.042274,-0.000531,-0.025177,-0.025730,-0.030648,-0.016472,-0.027330,0.032695,-0.001768,0.006116,-0.008904,0.050653,-0.010952,-0.007194,-0.018085,-0.049698,0.008482,-0.016584,-0.039945,0.008781,-0.003764,-0.001783,...,0.005420,0.069683,-0.007154,-0.125480,0.016297,0.053856,-0.059930,-0.064618,-0.023876,-0.064458,-0.016769,0.091364,-0.139449,-0.042287,-0.053730,0.005880,0.012420,-0.030252,0.076496,-0.014310,-0.056411,-0.014962,0.008311,0.018551,-0.201534,0.001345,0.004868,0.001520,0.107258,0.106533,0.006339,0.031008,-0.009703,-0.027321,-0.014041,-0.030959,-0.022415,-0.022086,-0.012576,0.020606
1,1403.7138,0.017416,-0.065326,-0.005350,0.011330,-0.059585,0.013561,-0.025806,0.046607,-0.009489,0.006072,0.056326,0.030148,0.031944,0.021936,0.008136,-0.012914,-0.021111,0.048298,0.010481,0.005039,0.001667,0.007354,0.019298,0.004105,0.001155,-0.006795,-0.001424,0.010394,0.009537,0.006879,-0.001400,0.004880,0.001531,-0.009286,-0.002389,0.033047,-0.020034,0.006441,0.011188,...,-0.025143,-0.053273,0.053872,-0.022409,-0.032081,-0.024660,-0.001255,0.034079,-0.050565,0.043055,0.031418,0.004060,0.006553,0.063791,0.044858,-0.018906,-0.020528,0.000528,0.022580,-0.039969,-0.003958,-0.020788,-0.012126,0.012905,0.003238,0.007290,0.168955,-0.143630,-0.047562,-0.023402,-0.012295,0.027360,-0.017133,0.047482,-0.047766,0.008870,0.056183,0.067945,0.022838,0.017862
2,1405.5857,0.035972,-0.014313,0.024727,-0.002210,0.038460,-0.041943,0.005480,-0.016978,-0.013649,0.009739,0.003327,0.026520,-0.017472,-0.007434,0.001166,-0.012303,0.004495,-0.015865,-0.015668,0.006241,0.016396,-0.000110,0.002951,-0.003692,-0.008267,0.005770,-0.000035,-0.014247,0.004500,-0.004276,0.002478,-0.000211,-0.006015,-0.018001,-0.018378,0.022288,-0.000473,0.000130,-0.001520,...,0.006040,-0.006189,0.022209,0.021526,-0.009381,-0.014020,0.013280,-0.011525,0.019687,-0.017334,0.051596,0.014490,0.035619,-0.006435,0.020131,-0.028405,0.030227,0.012058,0.000188,-0.021680

In [ ]:
ALL_pca_fastText.to_csv(OUTDIR / 'ALL_pca_fastText.csv', index=False)

## SciBERT

In [ ]:
class BertSequenceVectorizer:
    def __init__(self):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.model_name = 'scibert_scivocab_uncased' # modelを保存しているpathを指定
        self.tokenizer = BertTokenizer.from_pretrained(self.model_name)
        self.bert_model = transformers.BertModel.from_pretrained(self.model_name)
        self.bert_model = self.bert_model.to(self.device)
        self.max_len = 128
        print(self.device)


    def vectorize(self, sentence : str) -> np.array:
        inp = self.tokenizer.encode(sentence)
        len_inp = len(inp)

        if len_inp >= self.max_len:
            inputs = inp[:self.max_len]
            masks = [1] * self.max_len
        else:
            inputs = inp + [0] * (self.max_len - len_inp)
            masks = [1] * len_inp + [0] * (self.max_len - len_inp)

        inputs_tensor = torch.tensor([inputs], dtype=torch.long).to(self.device)
        masks_tensor = torch.tensor([masks], dtype=torch.long).to(self.device)

        bert_out = self.bert_model(inputs_tensor, masks_tensor)
        seq_out, pooled_out = bert_out['last_hidden_state'], bert_out['pooler_output']

        if torch.cuda.is_available():    
            return seq_out[0][0].cpu().detach().numpy()
        else:
            return seq_out[0][0].detach().numpy()

BSV = BertSequenceVectorizer()

cuda


In [ ]:
ALL_svd_SciBERT = pd.DataFrame(ALL_df["id"])
ALL_svd_SciBERT

,id
0,hep-ph/9902295
1,1403.7138
2,1405.5857
3,1807.01034
4,1905.05921
...,...
910603,1210.4112
910604,1701.03465
910605,1709.10428
910606,gr-qc/9803020


In [ ]:
with timer("ALL SVD abstract bert"):
    bert = ALL_df['abstract'].fillna('nan').progress_apply(lambda x: BSV.vectorize(x))
    svd = TruncatedSVD(n_components=32, random_state=SEED+1)
    X = np.stack(bert.values)
    X = svd.fit_transform(X)
    X_df = pd.DataFrame(X, columns=[f"ALL_scibert_abstract_{i}" for i in range(X.shape[1])])
    X_df["id"] = ALL_df["id"]
    ALL_svd_SciBERT = ALL_svd_SciBERT.merge(X_df, on="id", how="left")

  0%|          | 0/910608 [00:00<?, ?it/s]

[ALL SVD abstract bert] start


100%|██████████| 910608/910608 [3:53:05<00:00, 65.11it/s]


[ALL SVD abstract bert] done in 14011 s


In [ ]:
with timer("ALL SVD title bert"):
    bert = ALL_df['title'].fillna('nan').progress_apply(lambda x: BSV.vectorize(x))
    svd = TruncatedSVD(n_components=16, random_state=SEED+1)
    X = np.stack(bert.values)
    X = svd.fit_transform(X)
    X_df = pd.DataFrame(X, columns=[f"ALL_scibert_title_{i}" for i in range(X.shape[1])])
    X_df["id"] = ALL_df["id"]
    ALL_svd_SciBERT = ALL_svd_SciBERT.merge(X_df, on="id", how="left")

  0%|          | 8/910608 [00:00<3:12:02, 79.03it/s]

[ALL SVD title bert] start


100%|██████████| 910608/910608 [3:04:41<00:00, 82.17it/s]


[ALL SVD title bert] done in 11109 s


In [ ]:
ALL_svd_SciBERT.to_csv(OUTDIR / 'ALL_svd_SciBERT.csv', index=False)

# df Features

citesのあるtrain及びtestのみを使用した特徴量作成

## Word2Vec

In [ ]:
def make_id_feat(x):
    try: return re.search('(.*)/(.*)', x).group(1)
    except: return x[:4]
def make_doi_feat(x):
    try: return x.split('/')[0]
    except: return np.nan

df['id_feat'] = df['id'].apply(make_id_feat)
df['doi_feat'] = df['doi'].apply(make_doi_feat)
df['categories_space_list'] = df['categories'].apply(lambda x: list(x.split()))
df['categories_comma_list'] = df['categories'].apply(lambda x: list(re.split('[. ]', x)))
df.authors_parsed = df.authors_parsed.apply(lambda x: [[y[0]+' '+y[1], y[-1]] for y in x])
df['authors_list'] = df.authors_parsed.apply(lambda x: [i for i in list(itertools.chain.from_iterable(x)) if i!=''])

In [ ]:
id_df = df[['id', 'id_feat']].fillna(' ').rename(columns={'id_feat': 'target'})
doi_df = df[['id', 'doi_feat']].fillna(' ').rename(columns={'doi_feat': 'target'})
submitter_df = df[['id', 'submitter']].fillna(' ').rename(columns={'submitter': 'target'})
categories_df = df[['id', 'categories']].fillna(' ').rename(columns={'categories': 'target'})
authors_df = df[['id', 'authors_list']].fillna(' ').rename(columns={'authors_list': 'target'})
id_doi_df = pd.concat([df['id'],(df['id_feat'] + ' ' +  df['doi_feat']).apply(lambda x: list(x.split()))],axis=1).fillna(' ').rename(columns={0: 'target'})
categories_space_df = pd.concat([df['id'],df['categories_space_list']],axis=1).fillna(' ').rename(columns={'categories_space_list': 'target'})
categories_comma_df = pd.concat([df['id'],df['categories_comma_list']],axis=1).fillna(' ').rename(columns={'categories_comma_list': 'target'})
id_doi_categories_space_df = pd.concat([df['id'],id_doi_df['target'] + categories_space_df['target']],axis=1).fillna(' ').rename(columns={0: 'target'})

In [ ]:
model_size = {
    'id': 10,
    'doi': 10,
    'submitter': 10,
    'categories': 10,
    'authors': 10,
    'id_doi': 10,
    'categories_space': 10,
    'categories_comma': 15,
    'id_doi_categories_space': 30,
}

n_iter = 100
w2v_dfs = []
for _df, _df_name in zip(
        [id_df, doi_df, submitter_df, categories_df, authors_df, id_doi_df, categories_space_df, categories_comma_df, id_doi_categories_space_df],
        ['id', 'doi', 'submitter', 'categories', 'authors', 'id_doi', 'categories_space', 'categories_comma', 'id_doi_categories_space']
    ):

    with timer(f"Creating w2v for {_df_name}"):
        # Word2Vecの学習
        w2v_model = word2vec.Word2Vec(_df['target'].values.tolist(),
                                    size=model_size[_df_name],
                                    min_count=1,
                                    window=100,
                                    workers=1,
                                    iter=n_iter)

    with timer(f"Getting document vector for {_df_name}"):
        # 各文章ごとにそれぞれの単語をベクトル表現に直し、平均をとって文章ベクトルにする
        sentence_vectors = _df['target'].progress_apply(lambda x: np.mean([w2v_model.wv[e] for e in x], axis=0))
        sentence_vectors = np.vstack([x for x in sentence_vectors])
        sentence_vector_df = pd.DataFrame(sentence_vectors,
                                        columns=[f"{_df_name}_w2v_{i}" for i in range(model_size[_df_name])])
        sentence_vector_df.index = _df['id']
        w2v_dfs.append(sentence_vector_df)

[Creating w2v for id] start


  5%|▍         | 3655/74201 [00:00<00:01, 36538.54it/s]

[Creating w2v for id] done in 17 s
[Getting document vector for id] start


100%|██████████| 74201/74201 [00:02<00:00, 34877.99it/s]


[Getting document vector for id] done in 2 s
[Creating w2v for doi] start


...

[Getting document vector for categories_comma] done in 2 s
[Creating w2v for id_doi_categories_space] start


  6%|▌         | 4459/74201 [00:00<00:01, 44585.68it/s]

[Creating w2v for id_doi_categories_space] done in 34 s
[Getting document vector for id_doi_categories_space] start


100%|██████████| 74201/74201 [00:01<00:00, 42353.09it/s]


[Getting document vector for id_doi_categories_space] done in 2 s


In [ ]:
w2v = df[['id']]
for i in range(9):
    w2v = pd.merge(w2v, w2v_dfs[i], on='id', how='left')

In [ ]:
w2v.to_csv(OUTDIR / 'w2v.csv', index=False)

## fastText features

fastTextの学習済モデルを用いた特徴量作成

In [ ]:
model_en = load_model("cc.en.300.bin")

In [ ]:
def text_cleaning(raw_text):
    clean_text = hero.clean(raw_text, pipeline=[
        hero.preprocessing.fillna, # 欠損埋め
        hero.preprocessing.lowercase, # 小文字への統一
        hero.preprocessing.remove_digits, # 数字の削除
        hero.preprocessing.remove_punctuation, # 句読点の削除
        hero.preprocessing.remove_diacritics, # ダイアクリティカルマーク（発音区別符号。àやéなど）の削除
        hero.preprocessing.remove_stopwords, # ストップワードの除去
        hero.preprocessing.remove_whitespace, # スペースの削除
        hero.preprocessing.remove_brackets,
    ])
    return clean_text

In [ ]:
df['abstract'] = text_cleaning(df['abstract'])
df['title'] = text_cleaning(df['title'])
df['comments'] = text_cleaning(df['comments'])

In [ ]:
svd_fastText = pd.DataFrame(df["id"])
pca_fastText = pd.DataFrame(df["id"])
svd_fastText

,id
0,1403.7138
1,1405.5857
2,1807.01034
3,astro-ph/9908243
4,hep-ph/0103252
...,...
74196,1210.4112
74197,1701.03465
74198,1709.10428
74199,gr-qc/9803020


### TruncatedSVD

In [ ]:
with timer("SVD abstract fastText"):
    X = df["abstract"].progress_apply(lambda x: model_en.get_sentence_vector(x.replace("\n", "")))
    X = np.stack(X.values)
    svd = TruncatedSVD(n_components=64, random_state=SEED)
    X = svd.fit_transform(X)
    X_df = pd.DataFrame(X, columns=[f"SVD_abstract_fastText_{i}" for i in range(X.shape[1])])
    X_df["id"] = df["id"]
    svd_fastText = svd_fastText.merge(X_df, on="id", how="left")

  1%|          | 572/74201 [00:00<00:26, 2770.34it/s]

[SVD abstract fastText] start


100%|██████████| 74201/74201 [00:23<00:00, 3120.89it/s]


[SVD abstract fastText] done in 26 s


In [ ]:
with timer("SVD title fastText"):
    X = df["title"].progress_apply(lambda x: model_en.get_sentence_vector(x.replace("\n", "")))
    X = np.stack(X.values)
    svd = TruncatedSVD(n_components=32, random_state=SEED)
    X = svd.fit_transform(X)
    X_df = pd.DataFrame(X, columns=[f"SVD_title_fastText_{i}" for i in range(X.shape[1])])
    X_df["id"] = df["id"]
    svd_fastText = svd_fastText.merge(X_df, on="id", how="left")

  6%|▌         | 4468/74201 [00:00<00:03, 23236.84it/s]

[SVD title fastText] start


100%|██████████| 74201/74201 [00:03<00:00, 19086.81it/s]


[SVD title fastText] done in 5 s


In [ ]:
with timer("SVD comments fastText"):
    X = df["comments"].progress_apply(lambda x: model_en.get_sentence_vector(x.replace("\n", "")))
    X = np.stack(X.values)
    svd = TruncatedSVD(n_components=16, random_state=SEED)
    X = svd.fit_transform(X)
    X_df = pd.DataFrame(X, columns=[f"SVD_comments_fastText_{i}" for i in range(X.shape[1])])
    X_df["id"] = df["id"]
    svd_fastText = svd_fastText.merge(X_df, on="id", how="left")

  3%|▎         | 2149/74201 [00:00<00:03, 21476.96it/s]

[SVD comments fastText] start


100%|██████████| 74201/74201 [00:02<00:00, 29335.04it/s]


[SVD comments fastText] done in 4 s


In [ ]:
svd_fastText

,id,SVD_abstract_fastText_0,SVD_abstract_fastText_1,SVD_abstract_fastText_2,SVD_abstract_fastText_3,SVD_abstract_fastText_4,SVD_abstract_fastText_5,SVD_abstract_fastText_6,SVD_abstract_fastText_7,SVD_abstract_fastText_8,SVD_abstract_fastText_9,SVD_abstract_fastText_10,SVD_abstract_fastText_11,SVD_abstract_fastText_12,SVD_abstract_fastText_13,SVD_abstract_fastText_14,SVD_abstract_fastText_15,SVD_abstract_fastText_16,SVD_abstract_fastText_17,SVD_abstract_fastText_18,SVD_abstract_fastText_19,SVD_abstract_fastText_20,SVD_abstract_fastText_21,SVD_abstract_fastText_22,SVD_abstract_fastText_23,SVD_abstract_fastText_24,SVD_abstract_fastText_25,SVD_abstract_fastText_26,SVD_abstract_fastText_27,SVD_abstract_fastText_28,SVD_abstract_fastText_29,SVD_abstract_fastText_30,SVD_abstract_fastText_31,SVD_abstract_fastText_32,SVD_abstract_fastText_33,SVD_abstract_fastText_34,SVD_abstract_fastText_35,SVD_abstract_fastText_36,SVD_abstract_fastText_37,SVD_abstract_fastText_38,...,SVD_title_fastText_8,SVD_title_fastText_9,SVD_title_fastText_10,SVD_title_fastText_11,SVD_title_fastText_12,SVD_title_fastText_13,SVD_title_fastText_14,SVD_title_fastText_15,SVD_title_fastText_16,SVD_title_fastText_17,SVD_title_fastText_18,SVD_title_fastText_19,SVD_title_fastText_20,SVD_title_fastText_21,SVD_title_fastText_22,SVD_title_fastText_23,SVD_title_fastText_24,SVD_title_fastText_25,SVD_title_fastText_26,SVD_title_fastText_27,SVD_title_fastText_28,SVD_title_fastText_29,SVD_title_fastText_30,SVD_title_fastText_31,SVD_comments_fastText_0,SVD_comments_fastText_1,SVD_comments_fastText_2,SVD_comments_fastText_3,SVD_comments_fastText_4,SVD_comments_fastText_5,SVD_comments_fastText_6,SVD_comments_fastText_7,SVD_comments_fastText_8,SVD_comments_fastText_9,SVD_comments_fastText_10,SVD_comments_fastText_11,SVD_comments_fastText_12,SVD_comments_fastText_13,SVD_comments_fastText_14,SVD_comments_fastText_15
0,1403.7138,0.363249,0.036163,-0.057263,-0.004489,0.012805,-0.059323,-0.028795,-0.013458,-0.046762,0.015787,0.009254,0.057568,-0.016350,-0.032146,0.022353,-0.029111,0.002450,0.006109,0.008820,-0.045247,-0.000790,-0.007173,-0.000417,0.022783,0.010546,0.004842,-0.000400,0.005816,0.009108,0.001712,0.011019,0.002084,0.007094,-0.005201,0.001718,0.019941,0.034087,0.002584,0.000305,...,0.027592,0.000116,-0.025004,-0.075135,-0.034174,-0.007044,-0.016533,0.000785,-0.032443,-0.055686,0.039786,0.034201,-0.006243,0.001384,-0.039855,-0.053504,-0.008725,-0.026833,-0.001316,0.017851,-0.031920,-0.026893,0.023250,0.008686,0.463168,0.174357,0.018888,0.174098,-0.039492,-0.018320,-0.015919,0.026770,-0.016106,0.048573,-0.044592,0.006404,0.052530,0.079725,0.011259,0.018703
1,1405.5857,0.349707,0.036987,-0.007622,0.033312,-0.007483,0.033859,0.043349,-0.009825,0.014303,0.011604,0.010482,0.007153,-0.025723,0.018025,-0.008113,0.001240,-0.013418,-0.006776,0.010874,0.011695,0.012848,0.014915,-0.003451,0.003813,-0.005019,-0.009489,0.004690,-0.004948,-0.014124,0.005673,-0.000462,-0.001560,0.000566,-0.010549,0.017704,-0.000235,0.027379,-0.008552,-0.000534,...,-0.004573,0.009838,0.006643,-0.026310,0.015480,-0.004861,-0.015985,0.008354,0.011801,0.024794,0.003222,0.053409,0.002897,-0.038642,0.009655,-0.014963,-0.026050,0.032809,0.016913,-0.004278,-0.032901,0.017529,0.032499,0.010253,0.266957,0.204622,0.039873,-0.095879,-0.001073,0.060874,-0.052167,-0.006817,-0.016820,-0.010484,-0.013713,0.035832,0.012730,-0.027548,0.010707,-0.018787
2,1807.01034,0.362031,-0.002213,-0.035914,-0.016621,-0.020911,0.011678,0.011961,-0.008798,0.004393,-0.017695,-0.000504,-0.021193,0.026756,-0.012224,0.007477,0.020000,0.019287,0.025180,0.005475,0.000524,0.019232,-0.006324,0.000448,-0.022061,-0.009714,-0.006328,-0.039916,-0.003471,-0.000698,-0.006873,0.003015,0.000718,-0.009903,0.027684,0.005380,-0.005478,-0.004694,-0.007034,0.001672,...,0.044675,-0.014755,-0.014650,0.040130,0.053732,-0.035711,-0.016279,-0.026685,-0.029782,-0.046798,-0.042801,0.033379,-0.008264,0.083723,-0.079690,0.024421,-0.008615,0.093962,-0.012229,0.018197,-0.00

In [ ]:
svd_fastText.to_csv(OUTDIR / 'svd_fastText.csv', index=False)

### PCA

In [ ]:
with timer("PCA abstract fastText"):
    X = df["abstract"].progress_apply(lambda x: model_en.get_sentence_vector(x.replace("\n", "")))
    X = np.stack(X.values)
    pca = PCA(n_components=64, random_state=SEED)
    X = pca.fit_transform(X)
    X_df = pd.DataFrame(X, columns=[f"PCA_abstract_fastText_{i}" for i in range(X.shape[1])])
    X_df["id"] = df["id"]
    pca_fastText = pca_fastText.merge(X_df, on="id", how="left")

  1%|          | 606/74201 [00:00<00:24, 3019.93it/s]

[PCA abstract fastText] start


100%|██████████| 74201/74201 [00:24<00:00, 3052.77it/s]


[PCA abstract fastText] done in 26 s


In [ ]:
with timer("PCA title fastText"):
    X = df["title"].progress_apply(lambda x: model_en.get_sentence_vector(x.replace("\n", "")))
    X = np.stack(X.values)
    pca = PCA(n_components=32, random_state=SEED)
    X = pca.fit_transform(X)
    X_df = pd.DataFrame(X, columns=[f"PCA_title_fastText_{i}" for i in range(X.shape[1])])
    X_df["id"] = df["id"]
    pca_fastText = pca_fastText.merge(X_df, on="id", how="left")

  4%|▍         | 3130/74201 [00:00<00:04, 15375.39it/s]

[PCA title fastText] start


100%|██████████| 74201/74201 [00:03<00:00, 20311.73it/s]


[PCA title fastText] done in 5 s


In [ ]:
with timer("PCA comments fastText"):
    X = df["comments"].progress_apply(lambda x: model_en.get_sentence_vector(x.replace("\n", "")))
    X = np.stack(X.values)
    pca = PCA(n_components=16, random_state=SEED)
    X = pca.fit_transform(X)
    X_df = pd.DataFrame(X, columns=[f"PCA_comments_fastText_{i}" for i in range(X.shape[1])])
    X_df["id"] = df["id"]
    pca_fastText = pca_fastText.merge(X_df, on="id", how="left")

  4%|▍         | 3331/74201 [00:00<00:02, 33304.87it/s]

[PCA comments fastText] start


100%|██████████| 74201/74201 [00:02<00:00, 29120.12it/s]


[PCA comments fastText] done in 4 s


In [ ]:
pca_fastText

,id,PCA_abstract_fastText_0,PCA_abstract_fastText_1,PCA_abstract_fastText_2,PCA_abstract_fastText_3,PCA_abstract_fastText_4,PCA_abstract_fastText_5,PCA_abstract_fastText_6,PCA_abstract_fastText_7,PCA_abstract_fastText_8,PCA_abstract_fastText_9,PCA_abstract_fastText_10,PCA_abstract_fastText_11,PCA_abstract_fastText_12,PCA_abstract_fastText_13,PCA_abstract_fastText_14,PCA_abstract_fastText_15,PCA_abstract_fastText_16,PCA_abstract_fastText_17,PCA_abstract_fastText_18,PCA_abstract_fastText_19,PCA_abstract_fastText_20,PCA_abstract_fastText_21,PCA_abstract_fastText_22,PCA_abstract_fastText_23,PCA_abstract_fastText_24,PCA_abstract_fastText_25,PCA_abstract_fastText_26,PCA_abstract_fastText_27,PCA_abstract_fastText_28,PCA_abstract_fastText_29,PCA_abstract_fastText_30,PCA_abstract_fastText_31,PCA_abstract_fastText_32,PCA_abstract_fastText_33,PCA_abstract_fastText_34,PCA_abstract_fastText_35,PCA_abstract_fastText_36,PCA_abstract_fastText_37,PCA_abstract_fastText_38,...,PCA_title_fastText_8,PCA_title_fastText_9,PCA_title_fastText_10,PCA_title_fastText_11,PCA_title_fastText_12,PCA_title_fastText_13,PCA_title_fastText_14,PCA_title_fastText_15,PCA_title_fastText_16,PCA_title_fastText_17,PCA_title_fastText_18,PCA_title_fastText_19,PCA_title_fastText_20,PCA_title_fastText_21,PCA_title_fastText_22,PCA_title_fastText_23,PCA_title_fastText_24,PCA_title_fastText_25,PCA_title_fastText_26,PCA_title_fastText_27,PCA_title_fastText_28,PCA_title_fastText_29,PCA_title_fastText_30,PCA_title_fastText_31,PCA_comments_fastText_0,PCA_comments_fastText_1,PCA_comments_fastText_2,PCA_comments_fastText_3,PCA_comments_fastText_4,PCA_comments_fastText_5,PCA_comments_fastText_6,PCA_comments_fastText_7,PCA_comments_fastText_8,PCA_comments_fastText_9,PCA_comments_fastText_10,PCA_comments_fastText_11,PCA_comments_fastText_12,PCA_comments_fastText_13,PCA_comments_fastText_14,PCA_comments_fastText_15
0,1403.7138,0.022213,-0.063580,-0.006930,0.012005,-0.060646,0.007012,-0.026887,-0.047010,0.013938,0.007185,0.059024,-0.014033,-0.032295,0.020724,0.016359,-0.007507,-0.018424,0.048894,-0.009012,0.007991,-0.002842,-0.006262,0.022443,0.005798,-0.002985,0.005018,0.006741,0.009631,0.000006,0.010181,0.003141,0.006641,-0.000299,0.006594,-0.019092,0.035545,0.001255,0.000645,0.009431,...,-0.015427,-0.056469,0.057619,-0.026314,0.027284,-0.026489,0.000808,0.032853,-0.054654,0.042696,-0.026532,0.008023,-0.001114,0.051280,-0.054159,-0.009021,-0.021708,-0.001483,0.020374,0.033468,-0.016812,0.027175,-0.000312,-0.008590,0.002440,0.005865,0.166830,-0.145452,-0.045220,-0.020820,-0.014590,0.026754,-0.015489,0.048165,-0.046970,0.005813,0.053057,0.078501,0.011781,0.018191
1,1405.5857,0.037169,-0.012214,0.024233,-0.002867,0.044837,-0.036380,0.003958,0.016378,0.013121,0.009504,0.008721,-0.025580,0.018166,-0.006840,0.001289,-0.011708,0.001809,-0.014148,0.015613,0.011453,0.013115,-0.001125,0.003983,-0.003748,-0.002661,-0.009839,-0.005478,-0.013711,0.006578,0.001089,-0.001652,0.000873,-0.007856,0.019372,0.000215,0.024720,-0.011748,-0.004984,-0.005072,...,0.007250,-0.004720,0.024331,0.017823,0.004571,-0.013848,0.013540,-0.011610,0.018792,-0.006821,-0.054149,0.016559,-0.037320,-0.005688,-0.022170,-0.018319,0.035826,0.017803,-0.000399,0.030720,0.019443,0.032302,0.008010,0.000793,-0.193191,0.032933,0.045226,0.093213,0.010066,0.060231,-0.054515,-0.006676,-0.017124,-0.009652,-0.014209,0.035692,0.012673,-0.028530,0.010638,-0.017865
2,1807.01034,-0.012415,-0.036621,-0.013441,-0.021336,0.014132,-0.009251,-0.003328,0.005234,-0.011307,0.002026,-0.023612,0.026777,-0.013517,0.008134,-0.015343,0.025941,0.026618,0.001923,0.006342,0.019214,-0.006920,-0.003280,-0.022798,-0.016680,-0.020302,0.030592,-0.004742,-0.002571,-0.008502,0.001463,0.000861,-0.010143,0.026225,-0.000043,0.004637,-0.006892,-0.007443,0.000335,-0.002657,...,-0.005449,0.008538,-0.047452,0.048375,-0.032910,-0.039609,-0.020476,0.027560,-0.048890,-0.039564,-0.015069,0.005120,0.084489,0.077087,0.029557,-0.003029,0.090141,-0.015419,0.022399,-0.002227,-0.0

In [ ]:
pca_fastText.to_csv(OUTDIR / 'pca_fastText.csv', index=False)

## SciBERT

In [ ]:
class BertSequenceVectorizer:
    def __init__(self):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.model_name = 'scibert_scivocab_uncased' # modelを保存しているpathを指定
        self.tokenizer = BertTokenizer.from_pretrained(self.model_name)
        self.bert_model = transformers.BertModel.from_pretrained(self.model_name)
        self.bert_model = self.bert_model.to(self.device)
        self.max_len = 128
        print(self.device)


    def vectorize(self, sentence : str) -> np.array:
        inp = self.tokenizer.encode(sentence)
        len_inp = len(inp)

        if len_inp >= self.max_len:
            inputs = inp[:self.max_len]
            masks = [1] * self.max_len
        else:
            inputs = inp + [0] * (self.max_len - len_inp)
            masks = [1] * len_inp + [0] * (self.max_len - len_inp)

        inputs_tensor = torch.tensor([inputs], dtype=torch.long).to(self.device)
        masks_tensor = torch.tensor([masks], dtype=torch.long).to(self.device)

        bert_out = self.bert_model(inputs_tensor, masks_tensor)
        seq_out, pooled_out = bert_out['last_hidden_state'], bert_out['pooler_output']

        if torch.cuda.is_available():    
            return seq_out[0][0].cpu().detach().numpy()
        else:
            return seq_out[0][0].detach().numpy()

BSV = BertSequenceVectorizer()

cuda


In [ ]:
svd_SciBERT = pd.DataFrame(df["id"])
svd_SciBERT

,id
0,1403.7138
1,1405.5857
2,1807.01034
3,astro-ph/9908243
4,hep-ph/0103252
...,...
74196,1210.4112
74197,1701.03465
74198,1709.10428
74199,gr-qc/9803020


In [ ]:
with timer("SVD abstract scibert"):
    bert = df['abstract'].fillna('nan').progress_apply(lambda x: BSV.vectorize(x))
    svd = TruncatedSVD(n_components=32, random_state=SEED+1)
    X = np.stack(bert.values)
    X = svd.fit_transform(X)
    X_df = pd.DataFrame(X, columns=[f"scibert_abstract_{i}" for i in range(X.shape[1])])
    X_df["id"] = df["id"]
    svd_SciBERT = svd_SciBERT.merge(X_df, on="id", how="left")

  0%|          | 7/74201 [00:00<17:41, 69.88it/s]

[SVD abstract scibert] start


100%|██████████| 74201/74201 [19:26<00:00, 63.63it/s]


[SVD abstract scibert] done in 1170 s


In [ ]:
with timer("SVD title scibert"):
    bert = df['title'].fillna('nan').progress_apply(lambda x: BSV.vectorize(x))
    svd = TruncatedSVD(n_components=16, random_state=SEED+1)
    X = np.stack(bert.values)
    X = svd.fit_transform(X)
    X_df = pd.DataFrame(X, columns=[f"scibert_title_{i}" for i in range(X.shape[1])])
    X_df["id"] = df["id"]
    svd_SciBERT = svd_SciBERT.merge(X_df, on="id", how="left")

  0%|          | 10/74201 [00:00<13:28, 91.79it/s]

[SVD title scibert] start


100%|██████████| 74201/74201 [15:24<00:00, 80.28it/s]


[SVD title scibert] done in 927 s


In [ ]:
svd_SciBERT.to_csv(OUTDIR / 'svd_SciBERT.csv', index=False)

## Tfidf features

精度上がらないので不使用

In [ ]:
tfidf_dfs = []
VOCAB_SIZE = 10000
COMPONENT_SIZE = 50
text_columns = ['title', 'abstract', 'comments']
for col in text_columns:
    with timer(f'working on {col}'):
        docs = df[col]
        tv = TfidfVectorizer(max_features=VOCAB_SIZE, analyzer='word', ngram_range=(1, 3))
        X = tv.fit_transform(docs)

        svd = TruncatedSVD(n_components=COMPONENT_SIZE, random_state=SEED)
        X = svd.fit_transform(X)
        _df = pd.DataFrame(X, columns=[f'tfidf_{col}_{i}' for i in range(COMPONENT_SIZE)])
        _df.index = df['id']
        tfidf_dfs.append(_df)

[working on title] start
[working on title] done in 5 s
[working on abstract] start
[working on abstract] done in 57 s
[working on comments] start
[working on comments] done in 4 s


In [ ]:
tfidf_dfs[0]

,tfidf_title_0,tfidf_title_1,tfidf_title_2,tfidf_title_3,tfidf_title_4,tfidf_title_5,tfidf_title_6,tfidf_title_7,tfidf_title_8,tfidf_title_9,tfidf_title_10,tfidf_title_11,tfidf_title_12,tfidf_title_13,tfidf_title_14,tfidf_title_15,tfidf_title_16,tfidf_title_17,tfidf_title_18,tfidf_title_19,tfidf_title_20,tfidf_title_21,tfidf_title_22,tfidf_title_23,tfidf_title_24,tfidf_title_25,tfidf_title_26,tfidf_title_27,tfidf_title_28,tfidf_title_29,tfidf_title_30,tfidf_title_31,tfidf_title_32,tfidf_title_33,tfidf_title_34,tfidf_title_35,tfidf_title_36,tfidf_title_37,tfidf_title_38,tfidf_title_39,tfidf_title_40,tfidf_title_41,tfidf_title_42,tfidf_title_43,tfidf_title_44,tfidf_title_45,tfidf_title_46,tfidf_title_47,tfidf_title_48,tfidf_title_49
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1403.7138,0.022092,-0.025197,0.001876,0.001340,-0.013368,-0.015695,0.010623,-0.003666,-0.008180,0.002093,-0.002974,-0.005010,-0.004291,-0.006800,0.008022,-0.003574,-0.006543,-0.006811,0.001579,-0.001033,0.000433,0.003499,-0.004929,0.005418,-0.001621,-0.014007,-0.007102,-0.006330,0.004639,-0.000343,-0.000698,-0.000189,-0.006186,0.006236,0.000338,-0.002472,0.006124,-0.009179,-0.014351,-0.004119,0.023604,0.000761,-0.010376,-0.006981,0.034924,0.030098,0.010955,-0.007343,0.000830,0.009619
1405.5857,0.039071,-0.015600,-0.011777,-0.006100,-0.021955,-0.012472,0.010274,-0.007363,-0.019584,0.019338,-0.013201,-0.007193,-0.019394,0.005822,-0.022268,-0.013668,-0.049980,0.000151,-0.003409,0.005417,-0.050395,-0.009596,-0.000275,-0.024614,-0.005751,-0.023067,-0.009088,0.019793,-0.051109,0.011580,-0.003654,0.003815,-0.014845,0.017690,-0.014772,0.001448,0.022097,-0.011567,0.024491,0.013943,-0.038399,-0.036524,0.072319,0.105217,-0.001384,0.055054,-0.024756,0.036734,-0.005273,-0.006620
1807.01034,0.064398,0.010299,-0.002961,-0.016136,-0.036859,-0.002454,-0.014653,0.005606,-0.046992,0.046362,0.010179,0.003106,-0.088866,0.147413,-0.000095,0.138788,0.090244,0.230015,-0.054889,0.006077,0.026439,0.012836,0.062635,0.067700,0.048007,-0.008884,-0.007613,-0.037485,0.000729,-0.000334,0.017643,-0.002840,-0.019006,-0.023163,0.001837,-0.014909,0.029141,-0.011661,-0.026074,-0.021488,-0.003926,-0.020361,-0.005310,-0.008551,-0.001246,-0.011700,0.014528,0.044758,0.003956,-0.000280
astro-ph/9908243,0.041271,-0.055143,-0.018558,0.002658,-0.089187,-0.087120,0.054102,-0.042897,0.111121,-0.034166,0.061036,0.017554,-0.037726,0.006297,0.008291,-0.004310,0.002430,-0.007016,-0.007481,-0.000015,0.013666,0.008857,-0.012984,0.016916,0.022556,-0.010352,-0.000571,0.006515,0.009366,-0.004062,0.006338,0.013250,0.013342,-0.002487,-0.014131,0.002107,0.000227,0.004397,0.006032,-0.012862,0.000117,0.015252,-0.000039,0.010219,-0.004081,0.002097,0.000914,0.005600,-0.009347,-0.001482
hep-ph/0103252,0.078460,-0.115212,0.097084,0.010201,0.025180,0.029005,-0.027669,-0.020705,-0.004142,-0.004497,0.010110,0.013843,-0.009440,-0.044549,-0.022563,0.007759,-0.006818,0.012014,0.004066,0.037335,0.003853,0.011135,0.012955,-0.022339,0.018941,-0.022067,-0.008790,-0.031957,0.018606,0.056578,0.026778,-0.117538,-0.005887,-0.009796,0.000734,-0.040185,-0.050068,0.054078,-0.070434,0.016771,-0.036722,-0.019203,0.076261,-0.032048,0.026092,-0.000374,-0.054850,0.052380,-0.014402,-0.011968
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1210.4112,0.040924,-0.023408,-0.000949,-0.009484,-0.031344,-0.006087,0.014426,0.002487,0.037970,0.066051,-0.072117,-0.022755,0.056696,-0.110857,-0.104536,0.073635,0.040779,0.076894,0.065806,0.007308,-0.065504,0.054936,-0.010204,-0.007389,-0.083515,-0.016871,-0.077354,0.097635,0.009339,-0.023994,0.032001,-0.000033,0.026592,0.028432,-0.026142,-0.024227,-0.016424,0.000985,0.027325,0.023699,0.018981,0.000121,0.032671,-0.010991,0.003375,-0.029798,-0.020322,-0.017541,0.047261,-0.023958
1701.03465,0.041688,-0.046463,-0.006669,0.002882,-0.049054,-0.055600

In [ ]:
tfidf_title_abstract_comments = df[['id']]
for i in range(3):
    tfidf_title_abstract_comments = pd.merge(tfidf_title_abstract_comments, tfidf_dfs[i], how='left', on='id')

In [ ]:
tfidf_title_abstract_comments.to_csv(OUTDIR / 'tfidf.csv', index=False)